In [1]:
!pip install medmnist
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/87.2 kB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.8 MB/s eta 0:00:00


  Preparing metadata (setup.py) ... done


  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=e1a8c7b01644283b9f3e519a59849a7ac4bc4e1660fc49535d443ed0e5849041
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from torch.utils.data import random_split
from medmnist import OCTMNIST, PneumoniaMNIST, RetinaMNIST,BreastMNIST

# Define the transformation pipeline
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean for RGB
                         [0.229, 0.224, 0.225])  # ImageNet std for RGB
])

# Custom Dataset class to handle merged datasets and apply offsets
class CustomDataset(Dataset):
    def __init__(self, datasets, offsets, transform=None):
        self.datasets = datasets
        self.offsets = offsets
        self.transform = transform
        self.merged_samples = self._merge_datasets()

    def _merge_datasets(self):
        samples = []
        for name, dataset in self.datasets.items():
            offset = self.offsets[name]
            samples.extend(self.offset_dataset(dataset, offset))
        return samples

    def offset_dataset(self, dataset, offset):
        new_samples = []
        for x, y in dataset:
            new_y = torch.tensor([y[0] + offset])
            new_samples.append((x, new_y))
        return new_samples

    def __len__(self):
        return len(self.merged_samples)

    def __getitem__(self, idx):
        x, y = self.merged_samples[idx]
        if self.transform:
            x = self.transform(x)
        return x, y

# Load the datasets
datasets = {
    'oct': OCTMNIST(split='train', transform=None, download=True),
    'pneu': PneumoniaMNIST(split='train', transform=None, download=True),
    'retina': RetinaMNIST(split='train', transform=None, download=True),
    'breast': BreastMNIST(split='train', transform=None, download=True)
}

# Set offsets to distinguish different classes in each dataset
offsets = {'oct': 0, 'pneu': 4, 'retina': 6, 'breast': 11}

# Create the custom dataset and apply transformations
custom_dataset = CustomDataset(datasets, offsets, transform=transform)

len1=len(custom_dataset)//2
len2=len(custom_dataset) - len1

first_half, second_half = random_split(custom_dataset, [len1, len2])
loader1 = DataLoader(first_half, batch_size=32, shuffle=True)
loader2 = DataLoader(second_half, batch_size=32, shuffle=True)

print(f"Train_loader1 size:{len(loader1)}")
print(f"Train_loader2 size:{len(loader2)}")
#Creating the val datasets 
test_datasets = {
    'oct': OCTMNIST(split='test', transform=None, download=True),
    'pneu': PneumoniaMNIST(split='test', transform=None, download=True),
    'retina': RetinaMNIST(split='test', transform=None, download=True),
    'breast': BreastMNIST(split='test', transform=None, download=True)
}

offsets = {'oct': 0, 'pneu': 4, 'retina': 6, 'breast': 11}

# Create evaluation dataset and loader
eval_dataset = CustomDataset(test_datasets, offsets, transform=transform)
eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False)
print(f"Eval_loader size:{len(eval_loader)}")

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
import os

# Enable synchronous CUDA errors
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


aux_factor=0.3
# Load pretrained VGG16 model
model = models.inception_v3(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(2048, 13)
model.AuxLogits.fc = torch.nn.Linear(768, 13)

for param in model.fc.parameters():
    param.requires_grad=True
for param in model.AuxLogits.fc.parameters():
    param.requires_grad=True

# 3. Move model to appropriate device (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Training Loop
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(loader1):
        if inputs.size(0) == 1:  # Skip batches with size 1
            continue
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.squeeze(dim=1)

        # Validate labels
        if labels.max() >= 13 or labels.min() < 0:
            print(f"Batch {i} labels: {labels}")
            raise ValueError("Labels must be in range [0, 12]")

        # Ensure correct dtype
        labels = labels.long()  # Force to torch.long if not already

        optimizer.zero_grad()
        outputs = model(inputs)

        #outputs has two auxlogits and logits 
        aux_logits_loss = criterion(outputs.aux_logits, labels)
        logits_loss=criterion(outputs.logits,labels)
        total_loss=aux_logits_loss*aux_factor+logits_loss
        total_loss.backward()
        optimizer.step()
        running_loss += total_loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(loader1):.4f}")
    torch.cuda.empty_cache()

# torch.save(model.state_dict(), '_model.pth')
# print("Model saved successfully.")

In [ ]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(loader2):  # Replace 'loader' with your DataLoader
        if inputs.size(0) == 1:  # Skip batches with size 1
            continue
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.squeeze(dim=1)

        # Validate labels
        if labels.max() >= 13 or labels.min() < 0:
            print(f"Batch {i} labels: {labels}")
            raise ValueError("Labels must be in range [0, 12]")

        # Ensure correct dtype
        labels = labels.long()  # Force to torch.long if not already

        optimizer.zero_grad()
        outputs = model(inputs)

        #outputs has two auxlogits and logits 
        aux_logits_loss = criterion(outputs.aux_logits, labels)
        logits_loss=criterion(outputs.logits,labels)
        total_loss=aux_logits_loss*aux_factor+logits_loss
        total_loss.backward()
        optimizer.step()
        running_loss += total_loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(loader2):.4f}")
    torch.cuda.empty_cache()
    
torch.save(model.state_dict(), 'inception_v3model.pth')
print("Model saved successfully.")

In [ ]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix

def evaluate_model(model, dataloader, device, num_classes=13):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for inputs, targets in dataloader:
            if inputs.size(0) == 1: 
                continue
            inputs, targets = inputs.to(device), targets.to(device).squeeze()  # Squeeze targets
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)  # Probabilities for AUC
            _, predicted = torch.max(outputs, dim=1)  # Predicted classes

            # Collect predictions, labels, and probabilities
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # Convert to numpy arrays
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    # Accuracy
    accuracy = accuracy_score(all_labels, all_preds)

    # F1 Score (macro average for multi-class)
    f1 = f1_score(all_labels, all_preds, average='macro')

    # Precision and Recall (macro average)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')

    # AUC (one-vs-rest for multi-class)
    # Convert labels to one-hot encoding for AUC calculation
    try:
        auc = roc_auc_score(all_labels, all_probs, multi_class='ovr', average='macro')
    except ValueError as e:
        print(f"AUC calculation failed: {e}")
        auc = None

    # Confusion Matrix
    conf_matrix = confusion_matrix(all_labels, all_preds)

    # Print results
    print(f"Evaluation Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score (Macro): {f1:.4f}")
    print(f"Precision (Macro): {precision:.4f}")
    print(f"Recall (Macro): {recall:.4f}")
    print(f"AUC (One-vs-Rest, Macro): {auc:.4f}" if auc is not None else "AUC: N/A")
    print(f"Confusion Matrix:\n{conf_matrix}")

    # Return all metrics as a dictionary
    metrics = {
        'accuracy': accuracy,
        'f1_score': f1,
        'precision': precision,
        'recall': recall,
        'auc': auc,
        'confusion_matrix': conf_matrix
    }
    return metrics

# Example usage
metrics = evaluate_model(model, eval_loader, device)

In [ ]:
#Evaluation phase
def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device).squeeze()
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)

    accuracy = correct / total
    print(f"Evaluation Accuracy: {accuracy:.4f}")
    return accuracy

evaluate_model(model,eval_loader,device)

In [ ]:
import zipfile
with zipfile.ZipFile('/kaggle/working/inception_v3model.pth', 'w') as zipf:
    zipf.write('/kaggle/working')

In [ ]:
import torch
import zipfile
from IPython.display import FileLink
import shutil
# Step 1: Save the model weights
torch.save(model.state_dict(), '/kaggle/working/inception_v3model.pth')

# Step 2: Zip the saved weights
zip_path = '/kaggle/working/inception_v3model.zip'  # Correct zip file name
with zipfile.ZipFile(zip_path, 'w') as zipf:
    zipf.write('/kaggle/working/inception_v3model.pth', arcname='inception_v3model.pth')

shutil.make_archive('inception_v3model.zip', 'zip', '/kaggle/working/inception_v3model.zip')

# Step 3: Provide a download link
FileLink(zip_path)
